In [1]:
from PIL import Image
import os
import glob
from IPython.display import display, HTML
import pandas as pd
import shutil
import time

def compress_image(image_path, max_size_kb=500):
    """
    壓縮圖片至指定大小以下
    :param image_path: 圖片路徑
    :param max_size_kb: 目標檔案大小（KB）
    :return: 壓縮結果資訊
    """
    # 創建輸出目錄
    output_dir = "compressed_images"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 生成輸出檔案路徑
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    
    # 打開圖片
    img = Image.open(image_path)
    
    # 獲取原始檔案大小
    original_size = os.path.getsize(image_path) / 1024
    
    # 轉換為RGB模式（如果是RGBA）
    if img.mode in ('RGBA', 'P'):
        img = img.convert('RGB')
    
    # 初始品質
    quality = 95
    # 初始大小調整比例
    scale = 1.0
    
    # 根據 PIL 版本選擇重採樣方法
    try:
        LANCZOS = Image.Resampling.LANCZOS
    except AttributeError:
        LANCZOS = Image.LANCZOS
    
    while True:
        # 調整圖片大小
        if scale < 1.0:
            new_width = int(img.width * scale)
            new_height = int(img.height * scale)
            resized_img = img.resize((new_width, new_height), LANCZOS)
        else:
            resized_img = img
        
        # 直接保存到輸出路徑
        resized_img.save(output_path, 'JPEG', quality=quality)
        
        # 檢查檔案大小
        size_kb = os.path.getsize(output_path) / 1024
        
        if size_kb <= max_size_kb or (quality <= 20 and scale <= 0.3):
            return {
                'filename': os.path.basename(image_path),
                'original_size': original_size,
                'compressed_size': size_kb,
                'compression_ratio': (1 - size_kb/original_size) * 100,
                'final_quality': quality,
                'final_scale': scale,
                'output_path': output_path
            }
            
        # 如果檔案仍然太大，先降低品質，再考慮縮小尺寸
        if quality > 20:
            quality -= 5
        else:
            scale *= 0.9

def process_images_in_folder(folder_path=".", max_size_kb=500):
    """
    處理指定資料夾中的所有圖片
    :param folder_path: 資料夾路徑，預設為目前資料夾
    :param max_size_kb: 目標檔案大小（KB）
    """
    # 清理或創建輸出目錄
    output_dir = "compressed_images"
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)
    
    image_files = []
    for ext in ('*.jpg', '*.jpeg', '*.png'):
        image_files.extend(glob.glob(os.path.join(folder_path, '**', ext), recursive=True))
    
    results = []
    total_files = len(image_files)
    
    print(f"找到 {total_files} 個圖片檔案")
    
    for idx, image_path in enumerate(image_files, 1):
        try:
            print(f"處理 {idx}/{total_files}: {os.path.basename(image_path)}")
            result = compress_image(image_path, max_size_kb)
            if result:
                results.append(result)
                print(f"成功壓縮: {result['filename']} " 
                      f"({result['original_size']:.2f}KB -> {result['compressed_size']:.2f}KB)")
        except Exception as e:
            print(f"處理 {image_path} 時發生錯誤: {e}")
            continue
    
    # 創建DataFrame顯示結果
    if results:
        df = pd.DataFrame(results)
        df = df.round(2)  # 四捨五入到小數點後兩位
        
        # 設定DataFrame的樣式
        styled_df = df.style.background_gradient(subset=['compression_ratio'], cmap='RdYlGn')
        
        display(HTML("<h3>壓縮結果摘要:</h3>"))
        display(styled_df)
        
        # 顯示總體統計
        total_original = df['original_size'].sum()
        total_compressed = df['compressed_size'].sum()
        total_saved = total_original - total_compressed
        
        summary = f"""
        <h4>總體統計:</h4>
        <ul>
        <li>處理的檔案總數: {len(results)}</li>
        <li>原始總大小: {total_original:.2f} KB</li>
        <li>壓縮後總大小: {total_compressed:.2f} KB</li>
        <li>節省空間: {total_saved:.2f} KB ({(total_saved/total_original*100):.2f}%)</li>
        </ul>
        <p>壓縮後的圖片已保存在 '{output_dir}' 資料夾中</p>
        """
        display(HTML(summary))
    else:
        print("沒有處理任何圖片")

# 執行壓縮
process_images_in_folder(".", 500)  # 第一個參數是資料夾路徑，第二個參數是目標大小（KB）

找到 32 個圖片檔案
處理 1/32: 15057.jpg
成功壓縮: 15057.jpg (787.22KB -> 475.46KB)
處理 2/32: 20220416_152104.jpg
成功壓縮: 20220416_152104.jpg (4777.90KB -> 499.71KB)
處理 3/32: 21476.jpg
成功壓縮: 21476.jpg (3244.89KB -> 472.26KB)
處理 4/32: 22237.jpg
成功壓縮: 22237.jpg (4401.42KB -> 441.03KB)
處理 5/32: 25052.jpg
成功壓縮: 25052.jpg (3166.79KB -> 443.02KB)
處理 6/32: 27146.jpg
成功壓縮: 27146.jpg (3477.02KB -> 451.73KB)
處理 7/32: 29826.jpg
成功壓縮: 29826.jpg (3361.05KB -> 494.37KB)
處理 8/32: 304.jpg
成功壓縮: 304.jpg (4592.73KB -> 440.94KB)
處理 9/32: 5598.jpg
成功壓縮: 5598.jpg (1003.25KB -> 451.94KB)
處理 10/32: 6131.jpg
成功壓縮: 6131.jpg (631.33KB -> 398.45KB)
處理 11/32: 南方綠椿象1.jpg
成功壓縮: 南方綠椿象1.jpg (3208.14KB -> 476.92KB)
處理 12/32: 大螟2.jpg
成功壓縮: 大螟2.jpg (304.65KB -> 307.66KB)
處理 13/32: 弄蝶(稻苞蟲).jpg
成功壓縮: 弄蝶(稻苞蟲).jpg (1393.83KB -> 479.00KB)
處理 14/32: 徒長病1.jpg
成功壓縮: 徒長病1.jpg (2710.62KB -> 451.77KB)
處理 15/32: 斜紋夜蛾2.jpg
成功壓縮: 斜紋夜蛾2.jpg (1485.42KB -> 484.36KB)
處理 16/32: 暮眼蝶屬2.jpg
成功壓縮: 暮眼蝶屬2.jpg (3391.10KB -> 450.72KB)
處理 17/32: 水稻水象鼻蟲4.jpg
成功壓縮: 

,filename,original_size,compressed_size,compression_ratio,final_quality,final_scale,output_path
0,15057.jpg,787.220000,475.460000,39.600000,20,1.000000,compressed_images\15057.jpg
1,20220416_152104.jpg,4777.900000,499.710000,89.540000,25,1.000000,compressed_images\20220416_152104.jpg
2,21476.jpg,3244.890000,472.260000,85.450000,30,1.000000,compressed_images\21476.jpg
3,22237.jpg,4401.420000,441.030000,89.980000,30,1.000000,compressed_images\22237.jpg
4,25052.jpg,3166.790000,443.020000,86.010000,30,1.000000,compressed_images\25052.jpg
5,27146.jpg,3477.020000,451.730000,87.010000,35,1.000000,compressed_images\27146.jpg
6,29826.jpg,3361.050000,494.370000,85.290000,35,1.000000,compressed_images\29826.jpg
7,304.jpg,4592.730000,440.940000,90.400000,20,0.810000,compressed_images\304.jpg
8,5598.jpg,1003.250000,451.940000,54.950000,90,1.000000,compressed_images\5598.jpg
9,6131.jpg,631.330000,398.450000,36.890000,95,1.000000,compressed_images\6131.jpg
